# Import Libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Create a Function

In [6]:
def NikkisoScraper(year,month,day):
    # prepare the input date
    year=str(year)[2:]
    if month<10:
        month='0'+str(month)
    else:
        month=str(month)
    if day<10:
        month='0'+str(day)
    else:
        day=str(day)
    input_date=year+'年'+month+'月'+day+'日'

    #open chrome in incognito mode
    options = webdriver.ChromeOptions()
    options.add_argument(' -- incognito')
    browser = webdriver.Chrome(chrome_options=options)

    # deal with the first "medical staff?" question
    browser.get('https://webmedical.nikkiso.co.jp/')

    # wait for browser to open for 10 sec
    timeout = 10
    try:
        WebDriverWait(browser, timeout).until(
        EC.visibility_of_element_located(
        (By.XPATH, '//*[@id="checker_box"]/div/div/ul/li[1]/a')
        )
        )
    except TimeoutException:
        print('Timed Out Waiting for page to load')
        browser.quit()

    # Click the yes button
    login_bt=browser.find_element_by_xpath('//*[@id="checker_box"]/div/div/ul/li[1]/a')
    login_bt.click()
    browser.implicitly_wait(3)

    # Get info
    # Go to production info section
    top_info = browser.find_element_by_id('topInfo')
    # Go to the list
    news_list=top_info.find_elements_by_css_selector('li')
    # Go through the list
    info_list=[]
    for news in news_list:
        date = news.find_elements_by_css_selector('p.date')[0].text.split('\n')[0]
        # Get URL and title if date == input date
        if date==input_date:
            # Get link and title
            link=news.find_element_by_css_selector('p>a').get_attribute('href')
            title=news.find_element_by_css_selector('p>a').text
            product_info=news.find_element_by_css_selector('p>span.tag').text
            # Check if it's info about new products
            if product_info=='製品情報':
                new_product=1
            else:
                new_product=0
            # Append the info to the list
            info_list.append([date,title,link,new_product])
    return info_list


# Run the Function

In [7]:
if __name__=='__main__':
    year=2019
    month=9
    day=25
    result=NikkisoScraper(year,month,day)
    print(result)

[['19年09月25日', 'Siシリーズ販売開始！', 'https://webmedical.nikkiso.co.jp/productinfo/si%e3%82%b7%e3%83%aa%e3%83%bc%e3%82%ba%e8%b2%a9%e5%a3%b2%e9%96%8b%e5%a7%8b%ef%bc%81', 1]]
